# Foundations of Computer Science - Final Project


##     CdLM Data Science - University of Milano-Bicocca

####       - Project conducted by: Anastasia Marzi, Diana Tenca and Federico Signoretta

In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
gPlay=pd.read_csv("C:/Users/feder/OneDrive/Desktop/FCS_project/googleplaystore.csv", sep=",")
gPlay_review=pd.read_csv("C:/Users/feder/OneDrive/Desktop/FCS_project/googleplaystore_user_reviews.csv", sep=",")

## 1. Convert the app size to a number

In [9]:
# Per prima cosa, utilizziamo la funzione compile per individuare un pattern attraverso un'espressione regolare che
# va a ricercare la parte numerica e il prefisso.

app_size=re.compile(r'(?P<value>\d+(\.\d+)?)(?P<prefix>[kKmMgG])')

# La seguente funzione sostituisce i caratteri 'K','M' e 'G', opportunamente normalizzati, con i loro corrispettivi numerici.

def pref_to_num(dim):
    if dim.upper()=='K':
        return 1000
    if dim.upper()=='M':
        return 1000000
    if dim.upper()=='G':
        return 1000000000

#Questa funzione, inizialmente, ricerca la corrispondenza del valore dato in input con quello di app_size. Se tale corrispondenza
#occorre, attraverso i gruppi definiti precedentemente e sfruttando la funzione pref_to_num, calcola il prodotto e otteniamo l'app 
#size in forma numerica
def converter(size):
    occ=app_size.search(size)
    if occ:
        dim = occ.group('prefix')
        q=pref_to_num(dim)
        v = float(occ.group('value'))
        return int(v*q)
    else:
        np.nan
        
gPlay['Size_new']=gPlay['Size'].apply(converter)
gPlay[['Size','Size_new']].head()

,Size,Size_new
0,19M,19000000.0
1,14M,14000000.0
2,8.7M,8700000.0
3,25M,25000000.0
4,2.8M,2800000.0


In [10]:
gPlay[gPlay['Size']=='1,000+']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_new
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,NaN


In [11]:
# Abbiamo deciso di cancellare la riga 10472 del dataset poichè particolarmente diversa dalle altre e poco significativa per
# i nostri obiettivi 
gPlay=gPlay.drop(10472)

# 2. Convert the number of installs to a number

In [12]:
# Utilizziamo la funzione replace per eliminare le virgole e il simbolo + all'interno della colonna 'Installs'
#e utilizziamo la funzione di pandas to_numeric per convertire le singole stringhe in numero.
gPlay['Installs_new']=gPlay['Installs'].str.replace(r'\,','', regex=True).str.replace(r'\+','',regex=True).apply(pd.to_numeric)

gPlay[['Installs','Installs_new']].head()

,Installs,Installs_new
0,"10,000+",10000
1,"500,000+",500000
2,"5,000,000+",5000000
3,"50,000,000+",50000000
4,"100,000+",100000


# 3. Transform "Varies with device" into a missing value

In [13]:
# Utilizziamo ancora la funzione replace per sostituire 'Varies with device' con NaN, in questo modo ci sarà possibile effettuare
#operazioni senza avere errori.
gPlay.replace('Varies with device', np.nan, inplace = True)

# 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 of 4.2)

In [14]:
#Utilizziamo lo stesso metodo dell'esercizio 1 per convertire le due colonne in un dotted number. L'unica differenza
#è che all'interno dell'espressione regolare utilizziamo * per indicare che il pattern all'interno della parentesi può ricorrere
# 0 o più volte (al contrario del ? che significa che ricorre solo 0 o 1 volta).
pattern=re.compile(r'(?P<version>\d+(\.\d+)*)')

def extracted_version(version):
    occ=pattern.search(version)
    if occ:
        return occ.group('version')
    else:
        return np.nan

gPlay['Current Ver_converted']=gPlay['Current Ver'].astype(str).apply(extracted_version)
gPlay[['Current Ver_converted','Current Ver']].head()

,Current Ver_converted,Current Ver
0,1.0.0,1.0.0
1,2.0.0,2.0.0
2,1.2.4,1.2.4
3,NaN,NaN
4,1.1,1.1


In [15]:
#Un esempio di trasformazione è il seguente:
print( "from: ", gPlay['Current Ver'][656], " \n to: ",gPlay['Current Ver_converted'][656])

from:  1.5-beta  
 to:  1.5


In [16]:
gPlay['Android Ver_converted']=gPlay['Android Ver'].astype(str).apply(extracted_version)
gPlay[['Android Ver','Android Ver_converted']].head()

,Android Ver,Android Ver_converted
0,4.0.3 and up,4.0.3
1,4.0.3 and up,4.0.3
2,4.0.3 and up,4.0.3
3,4.2 and up,4.2
4,4.4 and up,4.4


# 5. Remove the duplicates

In [17]:
#Come prima cosa, rimuoviamo tutte le righe completamente uguali.
gPlay.drop_duplicates(keep=False, inplace=True)
#gPlay.count()
# Abbiamo notato che alcune applicazioni hanno Categorie diverse, quindi decidiamo di non eliminare le righe che
#presentano categorie diverse, per non perdere informazioni che potrebbero rivelarsi importanti.

gPlay[gPlay['App']=='Farm Heroes Saga'].sort_values(['App','Reviews'],ascending=False)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_new,Installs_new,Current Ver_converted,Android Ver_converted
10186,Farm Heroes Saga,FAMILY,4.4,7615646,71M,"100,000,000+",Free,0,Everyone,Casual,"August 7, 2018",5.2.6,2.3 and up,71000000.0,100000000,5.2.6,2.3
1880,Farm Heroes Saga,GAME,4.4,7614415,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up,70000000.0,100000000,5.1.8,2.3
2011,Farm Heroes Saga,GAME,4.4,7614407,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up,70000000.0,100000000,5.1.8,2.3
1733,Farm Heroes Saga,GAME,4.4,7614271,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up,70000000.0,100000000,5.1.8,2.3
1695,Farm Heroes Saga,GAME,4.4,7614130,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up,70000000.0,100000000,5.1.8,2.3


In [18]:
# Convertiamo la colonna Review da stringa a numero.
gPlay['Reviews']=gPlay['Reviews'].apply(pd.to_numeric) 

In [19]:
# Eliminiamo i duplicati mantenendo le categorie diverse e la riga che presenta il numero di Reviews più alto.
gPlay=gPlay.sort_values(['App','Reviews'],ascending=False).drop_duplicates(subset=['App','Category'], keep='first')
gPlay[gPlay['App']=='Farm Heroes Saga'].sort_values(['App','Reviews'],ascending=False)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_new,Installs_new,Current Ver_converted,Android Ver_converted
10186,Farm Heroes Saga,FAMILY,4.4,7615646,71M,"100,000,000+",Free,0,Everyone,Casual,"August 7, 2018",5.2.6,2.3 and up,71000000.0,100000000,5.2.6,2.3
1880,Farm Heroes Saga,GAME,4.4,7614415,70M,"100,000,000+",Free,0,Everyone,Casual,"July 26, 2018",5.1.8,2.3 and up,70000000.0,100000000,5.1.8,2.3


In [20]:
#Notiamo che l'App con il più alto numero di recensioni è Facebook, seguita da WhatsApp e Instagram.
gPlay[['App','Reviews']].sort_values(['Reviews'],ascending=False).head()

,App,Reviews
2544,Facebook,78158306
3904,WhatsApp Messenger,69109672
2604,Instagram,66577446
382,Messenger – Text and Video Chat for Free,56646578
1879,Clash of Clans,44893888


# 6. For each category, compute the number of apps

In [21]:
pd.DataFrame(gPlay.groupby('Category')['App'].count()).head()

,App
Category,
ART_AND_DESIGN,64
AUTO_AND_VEHICLES,85
BEAUTY,53
BOOKS_AND_REFERENCE,221
BUSINESS,394


# 7. For each category, compute the average rating

In [22]:
pd.DataFrame(gPlay.groupby('Category')['Rating'].mean()).head()

,Rating
Category,
ART_AND_DESIGN,4.357377
AUTO_AND_VEHICLES,4.190411
BEAUTY,4.278571
BOOKS_AND_REFERENCE,4.346429
BUSINESS,4.078481


# 8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [23]:
#Utilizziamo la funzione 'split' per dividere la tupla all'interno della colonna 'Genres' nei diversi generi (in caso ce ne
#sia più di uno). 
split_genres = gPlay['Genres'].str.split(pat = ';', expand=True)
#Attraverso la funzione concat di pandas, concateniamo le due colonne trovate prima.
df_conc = pd.concat([split_genres[0],split_genres[1]]).drop_duplicates().sort_values().dropna()
df_genres = pd.DataFrame(df_conc, columns = ['Genres'])
df_genres.reset_index().drop(columns=['index']).head(7)

,Genres
0,Action
1,Action & Adventure
2,Adventure
3,Arcade
4,Art & Design
5,Auto & Vehicles
6,Beauty


In [24]:
df_genres.to_csv(r"C:/Users/feder/OneDrive/Desktop/FCS_project/df_genres.csv",sep=';')

In [25]:
# Creiamo due diversi dataframe in cui associamo ogni applicazione con il suo genere (o generi in caso ce ne sia più di uno)
#Usiamo axis=1 all'interno della funzione concat in modo tale che avvenga una concatenazione per colonne. Infine, utilizziamo
#la funzione append per unire i due dataframe in uno solo. 
app = pd.DataFrame(gPlay['App'])
col1 = pd.concat([app,split_genres[0]],axis=1)
col1.columns = ['App','Genres']
col2 = pd.concat([app,split_genres[1]],axis=1)
col2.columns = ['App','Genres']
app_genres = col1.append(col2).dropna()

#Ad esempio, "Pixel Draw" appartiene a più generi.
#app_genres.sort_values('App').head()
app_genres[app_genres['App'] == 'Pixel Draw - Number Art Coloring Book']

,App,Genres
4,Pixel Draw - Number Art Coloring Book,Art & Design
4,Pixel Draw - Number Art Coloring Book,Creativity


In [26]:
app_genres.to_csv(r"C:/Users/feder/OneDrive/Desktop/FCS_project/app_genres.csv",sep=';')

# 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [27]:
gPlay['G_0']=split_genres[0]
gPlay['G_1']=split_genres[1]

In [28]:
#Creiamo due dataframe attraverso la funzione get_dummies passando in input le colonne G_0 e G_1, che corrispondono alle 
#due colonne di split_genres. Abbiamo dovuto porre dtype=bool, per avere True e False come output. In seguito, concateniamo 
#fra di loro i due dataframe in un unico dataframe df_dum e in seguito lo uniamo al nostro dataset originale.

df_dum_0=pd.get_dummies(gPlay['G_0'],dtype=bool)
df_dum_1=pd.get_dummies(gPlay['G_1'],dtype=bool)
df_dum=pd.concat([df_dum_0,df_dum_1],axis=1)
gPlay=pd.concat([gPlay,df_dum],axis=1)
gPlay.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Trivia,Video Players & Editors,Weather,Word,Action & Adventure,Brain Games,Creativity,Education,Music & Video,Pretend Play
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,Entertainment,...,False,False,False,False,False,False,False,False,False,False
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,Tools,...,False,False,False,False,False,False,False,False,False,False
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,Social,...,False,False,False,False,False,False,False,False,False,False
4362,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,Lifestyle,...,False,False,False,False,False,False,False,False,False,False
6334,"뽕티비 - 개인방송, 인터넷방송, BJ방송",VIDEO_PLAYERS,NaN,414,59M,"100,000+",Free,0,Mature 17+,Video Players & Editors,...,False,True,False,False,False,False,False,False,False,False


In [29]:
print('Genres:',gPlay['Genres'][4],'\n','Art & Design:', gPlay['Art & Design'][4],
      '\n','Creativity:', gPlay['Creativity'][4])

Genres: Art & Design;Creativity 
 Art & Design: True 
 Creativity: True


In [30]:
gPlay[['App','Genres','Entertainment','Tools','Social','Lifestyle']].head(4)

,App,Genres,Entertainment,Tools,Social,Lifestyle
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment,True,False,False,False
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools,False,True,False,False
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",Social,False,False,True,False
4362,💎 I'm rich,Lifestyle,False,False,False,True


# 10.	For each genre, compute the average rating. What is the genre with highest average?

In [31]:
#Ripetiamo lo stesso procedimento dell'esercizio 8, utilizzando in questo caso la colonna Rating al posto di App.
#genres1 = gPlay['Genres'].str.split(pat = ';', expand=True)
rating = pd.DataFrame(gPlay['Rating'])
col1 = pd.concat([rating,split_genres[0]],axis=1)
col1.columns = ['Rating','Genres']
col2 = pd.concat([rating,split_genres[1]],axis=1)
col2.columns = ['Rating','Genres']
rating_genres = col1.append(col2).dropna()

pd.DataFrame(rating_genres.groupby('Genres')['Rating'].mean().sort_values(ascending = False).head(1))
#Ordinando in modo decrescende e selezionando il primo elemento, verifichiamo, quindi, che il genere con la media 
#del rating maggiore è Events.

,Rating
Genres,
Events,4.435556


# 11.	For each app, compute the approximate income, obtain as a product of number of installs and price.

In [32]:
#Anche in questo caso, con la stessa procedura utilizzata per la colonna 'Installs', abbiamo tolto il carattere $ dalla 
#colonna Price e abbiamo convertito da stringa a numerico. In seguito, abbiamo creato la nuova colonna Approximate Income, 
#facendo il prodotto fra le colonne Installs_new e Price_new.
gPlay['Price_new']=gPlay['Price'].str.replace('$','',regex=True).apply(pd.to_numeric)

gPlay['Approximate Income'] = gPlay['Installs_new']*gPlay['Price_new']
gPlay[['App','Approximate Income']].sort_values('Approximate Income',ascending = False).head()

,App,Approximate Income
2241,Minecraft,69900000.0
5351,I am rich,39999000.0
5356,I Am Rich Premium,19999500.0
4034,Hitman Sniper,9900000.0
7417,Grand Theft Auto: San Andreas,6990000.0


In [33]:
gPlay['Approximate Income'][4362]
print("Number of installs: ",gPlay['Installs_new'][4362],
      "\n Price: ",gPlay['Price_new'][4362])

Number of installs:  10000 
 Price:  399.99


In [34]:
print("Income: " ,gPlay['Approximate Income'][4362])

Income:  $399.99


# 12.	For each app, compute its minimum and maximum Sentiment_polarity

In [35]:
#In questo esercizio utilizziamo il secondo dataset che ci è stato fornito. Dopo aver raggruppato per applicazioni, calcoliamo
#il massimo e il minimo della Sentiment Polarity e mostriamo i nostri risultati in un dataframe complessivo.
sentiment_max = gPlay_review.groupby('App')['Sentiment_Polarity'].max()
sentiment_min = gPlay_review.groupby('App')['Sentiment_Polarity'].min()

sent_min=pd.DataFrame(sentiment_min)
sent_max=pd.DataFrame(sentiment_max)

pd.DataFrame(sent_min.merge(sent_max,left_on='App', right_on='App',suffixes=('_min', '_max')).dropna().head(10))

#gPlay_review[gPlay_review['App'] == 'Hot or Not - Find someone right now']

,Sentiment_Polarity_min,Sentiment_Polarity_max
App,,
10 Best Foods for You,-0.800000,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500,0.910000
11st,-1.000000,1.000000
1800 Contacts - Lens Store,-0.300000,0.838542
1LINE – One Line with One Touch,-0.825000,1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000,1.000000
21-Day Meditation Experience,-0.265625,0.587500
"2Date Dating App, Love and matching",-0.645833,1.000000
2GIS: directory & navigator,-0.375000,1.000000


# Part 2

# 1.	For each app, compute the average number of words in its reviews

In [36]:
#Creiamo inizialmente una funzione che ci ritorna il numero di parole all'interno di ogni Review nel caso in cui il valore non 
#sia nullo. Per fare questo, bisogna per prima cosa sostituire il carattere ' con lo spazio, in modo tale che parole come 
#"That's" vengano conteggiate come due. In seguito applichiamo la nostra funzione alla colonna delle Review e calcoliamo la media
#dopo aver raggruppato per applicazione.

def count_words(review):
    if review is not np.nan:
        return len(review.replace("'"," ").split(' '))
    else: 
        return 0

gPlay_review['Number_words'] = gPlay_review['Translated_Review'].apply(count_words)
#gPlay_review['Average_Word'] = gPlay_review.groupby('App')['Number_Words'].mean()
#gPlay_review

pd.DataFrame(gPlay_review.groupby('App')['Number_words'].mean()).head()


,Number_words
App,
10 Best Foods for You,6.58
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,6.30
11st,12.50
1800 Contacts - Lens Store,9.40
1LINE – One Line with One Touch,3.90


# 2.	For each app, compute its longest review

In [37]:
#Ordiniamo in modo discendente gli elementi del secondo dataset in base al nome delle App e al numero di parole contenute 
#nelle reviews, il quale si trova nella colonna calcolata nell'esercizio precedente. Successivamente, eliminiamo i duplicati e 
#teniamo solo il primo elemento per ogni App e in questo modo si riesce a ottenere, per ogni app, la review più lunga.

# Serve? col_max = pd.DataFrame(gPlay_review.groupby('App')['Number_words'].max())
gPlay_review.sort_values(['App','Number_words'],ascending=False).drop_duplicates('App', keep='first').sort_values(['Number_words'], ascending = False).head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Number_words
26472,Chrome Dev,"»TOO BRIGHT!… NIGHT MODE, PLEASE. HOW MANY REQ...",Negative,-0.077839,0.540961,363
6454,Amazon Kindle,7/26: I realized phone allows choose SD storag...,Positive,0.068323,0.462951,195
63408,Hopper - Watch & Book Flights,I got burnt putting trust Hopper predict best ...,Positive,0.158772,0.486842,189
54464,Golf GPS Rangefinder: Golf Pad,I love app. I third year using - first year fr...,Positive,0.278512,0.454643,178
49408,Free Hypnosis,"HE'S AMAZINGLY GIFTED, CARING, ENLIGHTENING, U...",Positive,0.322853,0.607366,167


# 3.	For each app, compute the ratio between the number of installs and the number of reviews

In [38]:
#Aggiungiamo una colonna al dataset originale contenente il rapporto richiesto, prestando attenzione al caso in cui 
#il numero di reviews risulti pari a 0 e quindi in rapporto pari a più o meno infinito. In tal caso, utilizzando la funzione 
#replace, il rapporto viene indicato come NaN. 

gPlay['Ratio_inst_rev'] = (gPlay['Installs_new']/gPlay['Reviews']).replace([np.inf, -np.inf], np.nan)

gPlay[['App','Installs_new','Reviews','Ratio_inst_rev']].head()

,App,Installs_new,Reviews,Ratio_inst_rev
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,1000000,11661,85.755939
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,10000,19,526.315789
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",1000000,22098,45.252964
4362,💎 I'm rich,10000,718,13.927577
6334,"뽕티비 - 개인방송, 인터넷방송, BJ방송",100000,414,241.545894


# 4.	Cluster the apps according to the major android version (the first two digits — e.g. for 4.0.3 the major version is 4.0)

In [39]:
# Esattamente come nei primi esercizi, utilizziamo la funzione compile per individuare i due pattern così da identificare la 
# versione principale per ogni riga del dataset. Successivamente, definiamo la funzione major_version la quale ricerca una 
# corrispondenza tra il parametro version, che viene dato in input, e re_android. Se viene riscontrata tale corrispondenza
# allora la funzione ritorna la versione principale sotto forma di dotted number.  

re_android=re.compile(r'(?P<major_ver>\d+(\.\d+)?)')

def major_version(version):
    occ=re_android.search(version)
    if occ:
        return float(occ.group('major_ver'))
    else:
        return np.nan

# Applichiamo la funzione major_version alla colonna 'Android Ver_converted' del dataset e, alla fine, creiamo un 
# dataset nel quale ad ogni versione principale corrisponde la lista delle App associate. 

gPlay['Android Major Version']=gPlay['Android Ver_converted'].astype(str).apply(major_version)
cluster_android=pd.DataFrame(gPlay.groupby('Android Major Version')['App'].apply(list))
cluster_android.head()

,App
Android Major Version,
1.0,"[Tamilnadu Electricity Info, Bible du Semeur-B..."
1.5,"[df, Wifi Connect Library, Trazado de tuberia ..."
1.6,"[X-Wing Squadron Builder, WiFi Tether Router, ..."
2.0,"[waut.ch!, meStudying: AP English Lit, iCard B..."
2.1,"[weather HD, ZOOKEEPER DX TouchEdition, X-ray ..."


# 5.	For each cluster, compute the average date and the last date of an update

In [40]:
#Grazie alla funzione pd.to_datetime di Pandas applicata sulla colonna 'Last Updated', convertiamo tutte le date, che
#precedentemente erano in formato stringa, nel formato datetime. In questo modo ci risulta possibile effettuare le operazioni 
# per il calcolo della media e del massimo sulle date, cosa che non poteva accadere se fossero rimaste delle stringhe.
gPlay['Date']=gPlay['Last Updated'].apply(pd.to_datetime).astype(np.int64)
pd.DataFrame(gPlay.groupby('Android Major Version')['Date'].mean().apply(pd.to_datetime)).head()

,Date
Android Major Version,
1.0,2016-09-21 00:00:00.000000000
1.5,2015-01-14 21:36:00.000000000
1.6,2015-11-11 08:04:08.275862016
2.0,2015-06-14 17:13:50.769230848
2.1,2015-05-14 22:10:54.545454592


In [41]:
pd.DataFrame(gPlay.groupby('Android Major Version')['Date'].max().apply(pd.to_datetime)).head()

,Date
Android Major Version,
1.0,2018-08-04
1.5,2018-08-04
1.6,2018-08-06
2.0,2018-07-09
2.1,2018-07-21


# 6.	Excluding the free apps, what is the content rating with highest average price?

In [42]:
# Per prima cosa, eliminiamo le App 'Free', corrispondenti a quelle con prezzo uguale a zero. 
#Successivamente, raggruppiamo in base al 'Content Rating' e calcoliamo la media del prezzo precedentemente modificato. 
#Per verificare a quale 'Content Rating' corrisponde il prezzo maggiore, come già fatto diverse volte, li ordiniamo 
#in ordine decrescente in base alla media e stampiamo il primo risultato, che in questo caso è Everyone.  

gPlay_pay =gPlay[gPlay['Price_new'] != 0]
pd.DataFrame(gPlay_pay.groupby('Content Rating')['Price_new'].mean().sort_values(ascending=False)).head(1)
#gPlay.groupby('Content Rating').count()
#gPlay[gPlay['Content Rating'] == 'Unrated']

,Price_new
Content Rating,
Everyone,14.874482


# Part 3

# 1.	What is the genre with the highest total income?

In [43]:
#Utilizzando lo stesso procedimento già visto nell'esercizio 8 e nel 10, isoliamo la colonna dell'Approximate Income e la
#mettiamo in relazione alle due colonne del dataframe split_genres, che ricordiamo contenere i diversi generi delle app. 
#In questo modo, dopo aver raggruppato per Genere e per mezzo della funzione aggregata sum, siamo riusciti a trovare il genere 
#con l'income totale più alto, cioè Action & Adventure.
income = pd.DataFrame(gPlay['Approximate Income'])
col1 = pd.concat([income,split_genres[0]],axis=1)
col1.columns = ['Income','Genres']
col2 = pd.concat([income,split_genres[1]],axis=1)
col2.columns = ['Income','Genres']
rating_genres = col1.append(col2).dropna()

pd.DataFrame(rating_genres.groupby('Genres')['Income'].sum().sort_values(ascending = False).head(1))

,Income
Genres,
Action & Adventure,77368035.0


# 2.	What is the genre with the highest fraction of free apps (over the number of all apps)?

In [44]:
#Per prima, cosa dobbiamo ovviamente contare il numero totale di app, per individuare il denominatore della frazione.
#In seguito, consideriamo solamente le applicazioni gratuite, al contrario dell'esercizio precedente.
#Riutilizzando nuovamente lo stesso metodo, associamo le app gratuite ai propri generi e poi contiamo il numero
#di queste applicazioni. Infine, creiamo un dataframe a parte (app_cost) in cui mettiamo assieme i risultati ottenuti (numero
#totale di app per genere e numero totale di app gratuite) e aggiungiamo la nuova colonna contentente il rapporto richiesto. 
#Ordinando in modo decrescente, otteniamo il genere con la più alta frazione di applicazioni gratuite. 
#Si noti che, in questo caso, ci sono 6 diversi generi che hanno solamente app gratuite e quindi non possiamo ottenere un 
#risultato assoluto.
app_number = pd.DataFrame(app_genres.groupby('Genres')['App'].count())
gPlay_free =gPlay[gPlay['Price_new'] == 0]

app_free = pd.DataFrame(gPlay_free['App'])
col1 = pd.concat([app_free,split_genres[0]],axis=1)
col1.columns = ['App','Genres']
col2 = pd.concat([app_free,split_genres[1]],axis=1)
col2.columns = ['App','Genres']
app_genres_free = col1.append(col2).dropna()

app_number_free = pd.DataFrame(app_genres_free.groupby('Genres')['App'].count())

app_cost = app_number.merge(app_number_free,left_on='Genres', right_on='Genres',suffixes=('_total', '_free')).dropna()
app_cost['Ratio_free_tot'] = app_cost['App_free']/app_cost['App_total']
app_cost['Ratio_free_tot'] = app_cost['Ratio_free_tot'].replace([np.inf, -np.inf], np.nan)
app_cost.sort_values('Ratio_free_tot',ascending=False).head(10)

,App_total,App_free,Ratio_free_tot
Genres,,,
Word,23,23,1.000000
Comics,56,56,1.000000
Trivia,39,39,1.000000
Music & Audio,1,1,1.000000
Beauty,53,53,1.000000
House & Home,67,67,1.000000
News & Magazines,243,241,0.991770
Shopping,183,181,0.989071
Libraries & Demo,84,83,0.988095


# 3.	For each rating, compute the average income

In [45]:
pd.DataFrame(gPlay.groupby('Rating')['Approximate Income'].mean())

,Approximate Income
Rating,
1.0,6.525000
1.2,0.000000
1.4,0.000000
1.5,0.000000
1.6,0.000000
1.7,62.375000
1.8,186.250000
1.9,0.000000
2.0,38.250000


# 4.	For each (Content Rating, Genre) pair, compute the number of reviews and the average rating.

In [46]:
#La caratteristica di questo esercizio è che dobbiamo concatenare 5 diverse colonne, ottenendo un dataframe 
#leggermente più grande, ma che contiene tutte le informazioni necessarie. Inoltre, dobbiamo raggruppare secondo una 
#coppia di parametri (Content Rating, Genres) e non solamente per un parametro, come accadeva negli esercizi precedenti.
app = pd.DataFrame(gPlay['App'])
content_rating = pd.DataFrame(gPlay['Content Rating'])
rating = pd.DataFrame(gPlay['Rating'])
review = pd.DataFrame(gPlay['Reviews'])
col1 = pd.concat([app,split_genres[0],content_rating,rating,review],axis=1)
col1.columns = ['App','Genres','Content Rating', 'Rating','Review']
col2 = pd.concat([app,split_genres[1],content_rating,rating,review],axis=1)
col2.columns = ['App','Genres','Content Rating', 'Rating','Review']
total = col1.append(col2).dropna()

In [47]:
#numero di recesioni
pd.DataFrame(total.groupby(['Content Rating','Genres'])['Review'].count()).head()

Review
Content Rating  Genres                    
Adults only 18+ Comics                   2
                Sports                   1
Everyone        Action                  73
                Action & Adventure      76
                Adventure               30

In [48]:
#punteggio medio
pd.DataFrame(total.groupby(['Content Rating','Genres'])['Rating'].mean()).head()

Rating
Content Rating  Genres                      
Adults only 18+ Comics              4.200000
                Sports              4.500000
Everyone        Action              4.208219
                Action & Adventure  4.286842
                Adventure           4.150000